# Protein Folding | EPOCH Lab 2022

In [1]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout

import pytraj as pt
import nglview as nv

In [2]:
try:
    platform = Platform.getPlatformByName("CUDA")
except Exception:
    platform = Platform.getPlatformByName("OpenCL")

In [11]:
pdb = PDBFile('pdb/.pdb')
forcefield = ForceField('amber99sb.xml', 'tip3p.xml')

In [4]:
unmatched_residues = forcefield.getUnmatchedResidues(pdb.topology)
# unmatched_residues

In [5]:
modeller = Modeller(pdb.topology, pdb.positions)
modeller.addHydrogens(forcefield)
modeller.addSolvent(forcefield)
modeller.topology

<Topology; 2 chains, 2798 residues, 8867 atoms, 6111 bonds>

In [6]:
system = forcefield.createSystem(modeller.topology,
                                 nonbondedMethod=PME,
                                 nonbondedCutoff=1*nanometer,
                                 constraints=HBonds)

In [7]:
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)

In [8]:
simulation = Simulation(modeller.topology, system, integrator, platform)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('/tmp/output.pdb', 10))
simulation.reporters.append(StateDataReporter(stdout, 10, step=True, potentialEnergy=True, temperature=True))

In [9]:
simulation.step(1000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
10,-145233.97391541512,7.418624702529998
20,-144809.78641541512,13.602530505092554
30,-144403.22391541512,19.10662126282485
40,-143972.69266541512,24.42451890741026
50,-143536.44266541512,29.48372092741491
60,-143123.88016541512,34.06906034135008
70,-142740.34891541512,39.723977766380116
80,-142307.03641541512,44.26622131957267
90,-141928.22391541512,49.37044578367001
100,-141540.88016541512,53.49304905020042
110,-141166.72391541512,58.4794114896611
120,-140865.22391541512,64.11448916803808
130,-140494.16141541512,68.43796217064124
140,-140139.31766541512,72.69361573408626
150,-139841.25516541512,77.55523260726693
160,-139480.38016541512,81.67039719869125
170,-139145.44266541512,85.60539973588982
180,-138954.25516541512,91.8524078520099
190,-138484.59891541512,94.69037185437404
200,-138105.84891541512,97.43550204559392
210,-137839.25516541512,102.07912400786371
220,-137528.00516541512,106.29947014511912
230,-137216.34891541512,109.

In [16]:
traj = pt.load('/tmp/output.pdb')

view = nv.show_pytraj(traj)
view.add_representation('licorice')
view

NGLWidget(max_frame=99)